In [1]:
import scipy.sparse as sp
import numpy as np
from py_accel.accel_solver import Solver
import discretize
import time
from pydiso.mkl_solver import MKLPardisoSolver

In [2]:
n = 100

mesh = discretize.TensorMesh((n, n))
D = mesh.face_divergence
Mf = mesh.get_face_inner_product()

In [3]:
A = D @ Mf @ D.T

x = np.random.rand(mesh.n_cells)

b = A @ x

In [7]:
t1 = time.time()
Ainv = Solver(A)
Ainv.factor()
t2 = time.time()
x2 = Ainv.solve(b)
t3 = time.time()
print("n:", n, 'nnz', A.nnz)
print("solve worked:", np.allclose(x, x2))
print("Accelerate factor:", t2-t1, "solve:", t3-t2)

n: 100 nnz 49600
solve worked: True
Accelerate factor: 0.024818897247314453 solve: 0.003463268280029297


In [8]:
t1 = time.time()
Ainv2 = MKLPardisoSolver(A, matrix_type='real_symmetric_positive_definite')
t2 = time.time()

x3 = Ainv2.solve(b)
t3 = time.time()
print("solve worked:", np.allclose(x, x3))
print("MKL Pardiso factor:", t2-t1, "solve:", t3-t2)

solve worked: True
MKL Pardiso factor: 0.04891395568847656 solve: 0.0009300708770751953


In [32]:
C = mesh.edge_curl
Me = mesh.get_edge_inner_product()
Mfz = sp.diags(mesh.cell_volumes)
print(C.shape, Mfz.shape, Me.shape)
A_c = C.T @ Mfz @ C - 1j * Me
A_r = C.T @ Mfz @ C
A_i = -Me

x_c = np.random.rand(A_c.shape[1]) + 1j * np.random.rand(A_c.shape[1])

b_c = A_c @ x_c

(10000, 20200) (10000, 10000) (20200, 20200)


$$
(A + i B)(x_r + i x_i) = (c_r + i c_i)
$$

$$
A x_r  - B x_i + i (A x_i + B x_r) = (c_r + i c_i)
$$

$$
A x_r - B x_i = c_r\\
A x_i + B x_r = c_i
$$

$$
\begin{pmatrix}
A & - B \\
B & A
\end{pmatrix}
\begin{pmatrix}
x_r\\
x_i
\end{pmatrix}
=
\begin{pmatrix}
c_r\\
c_i
\end{pmatrix}
$$
$$
\begin{pmatrix}
B & A\\
A & - B
\end{pmatrix}
\begin{pmatrix}
x_r\\
x_i
\end{pmatrix}
=
\begin{pmatrix}
c_i\\
c_r
\end{pmatrix}
$$


$$
\begin{pmatrix}
I & B^{-1}A\\
A & - B
\end{pmatrix}
\begin{pmatrix}
x_r\\
x_i
\end{pmatrix}
=
\begin{pmatrix}
B^{-1}c_i\\
c_r
\end{pmatrix}
$$




$$
\begin{pmatrix}
I & B^{-1}A\\
-B^{-1} A & I
\end{pmatrix}
\begin{pmatrix}
x_r\\
x_i
\end{pmatrix}
=
\begin{pmatrix}
B^{-1}c_i\\
-B^{-1} c_r
\end{pmatrix}
$$

In [36]:
A_solve = sp.bmat([[-A_i, A_r], [A_r, A_i]], format='csc')

t1 = time.time()
Ainv3 = Solver(A_solve)
Ainv3.factor()
t2 = time.time()

def solve_it(b):
    n = len(b)
    x = Ainv3.solve(np.r_[b.real, b.imag])
    return 1j * x[:n] + x[n:]

x_c2 = solve_it(b_c)
r = b_c - A_c @ x_c2
print(np.linalg.norm(r))
x_c2 += solve_it(r)
r = b_c - A_c @ x_c2
print(np.linalg.norm(r))
t3 = time.time()
print('Solver worked:', np.allclose(x_c, x_c2))
print(f'Apple time, factor: {t2-t1}, solve: {t3-t2}')

2.507044910689753e-09
3.427598238418928e-14
Solver worked: True
Apple time, factor: 0.21498799324035645, solve: 0.037648916244506836


In [34]:
t1 = time.time()
Ainv4 = MKLPardisoSolver(A_c, matrix_type='complex_symmetric')
t2 = time.time()
x_c3 = Ainv4.solve(b_c)
t3 = time.time()
print('Solver worked:', np.allclose(x_c, x_c3))
print(f'MKL time, factor: {t2-t1}, solve: {t3-t2}')

Solver worked: True
MKL time, factor: 0.11150693893432617, solve: 0.003846883773803711
